In [1]:
from __future__ import print_function
import sys
sys.path.append('..')
import os
import yaml
import argparse
from shutil import copyfile

from medseg import datagen, engine, utils

Using TensorFlow backend.


In [2]:
config_file = 'configs_a/config_segment_unet.yaml'
# Parser config file
with open(config_file) as f:
    config = yaml.load(f)

# Create experiment output directories
exp_dir = os.path.join(config['output_dir'], config['exp_id'])
utils.create_dirs([exp_dir,
        os.path.join(exp_dir, 'checkpoints'),
        os.path.join(exp_dir, 'logs')])

# Copy config file
copyfile(config_file, os.path.join(exp_dir, config['exp_id']+'.yaml'))

In [3]:
# Train and validation data generators
paths_train, annot_train, _, paths_val, annot_val, _ = datagen.get_paths(config['data_path'])
datagen_train = datagen.datagen_segment(paths_train, annot_train, **config['datagen'])
datagen_val = datagen.datagen_segment(paths_val, annot_val, **config['datagen'])

In [4]:
x,y = next(datagen_train)
print(x.shape, x.min(), x.max(), y.shape, y.min(), y.max())

(8, 320, 320, 2) 0.0 1.0 (8, 320, 320, 1) 0.0 1.0


In [5]:
# Tensorflow initialize session
utils.init_session()
    
# Define Classification model
model = engine.Model(**config['model'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 2)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 320, 320, 32) 608         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 320, 320, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 320, 320, 32) 128         activation_1[0][0]               
__________________________________________________________________________________________________
conv2d_2 (

In [6]:
model.train(datagen_train, datagen_val, exp_dir, **config['train'])

Epoch 1/2
 - 1964s - loss: 0.1301 - acc: 0.7903 - val_loss: 0.2218 - val_acc: 0.6371
Epoch 2/2
 - 1958s - loss: 0.1188 - acc: 0.7989 - val_loss: 2.4855e-05 - val_acc: 1.0000

Epoch 00002: saving model to ../output/debug_segment/checkpoints/wt-02-1.00.h5
